In [ ]:
# XGBoost + CatBoost Classification for Crocodile Species
# Baseline, Bayesian tuning with Optuna, Feature Engineering, Class Imbalance Handling, and Feature Importance

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix, classification_report

import xgboost as xgb
import optuna

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Load dataset from root CSV
DATA_PATH = "crocodile_dataset.csv"
raw_df = pd.read_csv(DATA_PATH)
raw_df.head()


In [ ]:
print("Shape:", raw_df.shape)
display(raw_df.head())
print("\nInfo:")
print(raw_df.info())

print("\nClass distribution (Common Name):")
print(raw_df['Common Name'].value_counts(normalize=True))

# Example feature distribution plots for a couple of numeric columns
numeric_cols = raw_df.select_dtypes(include=[np.number]).columns.tolist()
if numeric_cols:
    raw_df[numeric_cols].hist(figsize=(12, 8))
    plt.tight_layout()
    plt.show()

# Correlation heatmap for numerical features
if len(numeric_cols) <= 20 and len(numeric_cols) > 1:
    plt.figure(figsize=(10, 8))
    sns.heatmap(raw_df[numeric_cols].corr(), annot=False, cmap='coolwarm')
    plt.title('Correlation Heatmap (Numeric Features)')
    plt.show()


In [ ]:
# Preprocessing: remove contested species and leak features, then create train/val split

DF = raw_df.copy()

# Remove contested species to avoid taxonomic ambiguity
# Borneo Crocodile taxonomic status is disputed= excluded
CONTESTED_SPECIES_NAME = 'Borneo Crocodile (disputed)'
if CONTESTED_SPECIES_NAME in DF['Common Name'].unique():
    DF = DF[DF['Common Name'] != CONTESTED_SPECIES_NAME].copy()
    print(f"Removed contested species '{CONTESTED_SPECIES_NAME}', new shape: {DF.shape}")

# Drop columns that directly reveal the target (data leakage)
leak_cols = [col for col in ['Observation ID', 'Scientific Name', 'Family', 'Genus', 
                              'Conservation Status', 'Observer Name', 'Notes'] if col in DF.columns]
print("Dropping leak columns:", leak_cols)
DF = DF.drop(columns=leak_cols)


if 'Date of Observation' in DF.columns:
    DF['Date of Observation'] = pd.to_datetime(DF['Date of Observation'], dayfirst=True, errors='coerce')
    DF['Year'] = DF['Date of Observation'].dt.year
    DF['Month'] = DF['Date of Observation'].dt.month
    
    # Normalize year using full dataset statistics before train/test split
    if DF['Year'].notna().any():
        max_year = DF['Year'].max()
        min_year = DF['Year'].min()
        if max_year != min_year:
            DF['Year'] = (DF['Year'] - min_year) / (max_year - min_year)
    
    # Sine transformation = cyclic nature of months
    DF['Month'] = np.sin((np.pi * DF['Month']) / 12)
    DF = DF.drop(columns=['Date of Observation'])

# Define features and target (predicting Common Name)
target_col = 'Common Name'
X = DF.drop(columns=[target_col])
y = DF[target_col]

# Identify column types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print("Categorical columns:", categorical_cols)
print("Numeric columns:", numeric_cols)

# METHODOLOGY: Preprocessing pipeline design
# Numeric features: median imputation (robust to outliers) + standardization (mean=0, std=1)
# Categorical features: mode imputation + one-hot encoding
# StandardScaler ensures all numeric features have similar scales for tree-based models
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Median robust to outliers
    ('scaler', StandardScaler())  # Standardization for consistent feature scales
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Mode imputation for categoricals
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encoding
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Stratified train/validation split
# Stratification ensures class distribution is preserved in both sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y  # Preserve class distribution in train/val sets
)

print(f"Train shape: {X_train.shape}, Valid shape: {X_valid.shape}")
print(f"Number of classes: {len(y.unique())}")


In [ ]:
# Feature Engineering: Interaction Features and Target Encoding
# Create interaction features to capture non-linear relationships


X_train_fe = X_train.copy()
X_valid_fe = X_valid.copy()

# 1. Interaction Features
# Create multiplicative and ratio features to capture non-linear relationships
# These features may help models learn species-specific size patterns
print("Creating interaction features...")

# Length × Weight captures overall body size (larger crocodiles)
# Multiplicative interaction may reveal species-specific size relationships
if 'Observed Length (m)' in X_train_fe.columns and 'Observed Weight (kg)' in X_train_fe.columns:
    X_train_fe['Length_x_Weight'] = X_train_fe['Observed Length (m)'] * X_train_fe['Observed Weight (kg)']
    X_valid_fe['Length_x_Weight'] = X_valid_fe['Observed Length (m)'] * X_valid_fe['Observed Weight (kg)']

# Year × Month captures temporal interaction patterns
# May reveal seasonal or temporal trends in observation patterns
if 'Year' in X_train_fe.columns and 'Month' in X_train_fe.columns:
    X_train_fe['Year_x_Month'] = X_train_fe['Year'] * X_train_fe['Month']
    X_valid_fe['Year_x_Month'] = X_valid_fe['Year'] * X_valid_fe['Month']

# Weight/Length ratio indicates body condition (mass per unit length)
# Different species may have different body proportions
if 'Observed Length (m)' in X_train_fe.columns and 'Observed Weight (kg)' in X_train_fe.columns:
    X_train_fe['Weight_per_Length'] = X_train_fe['Observed Weight (kg)'] / (X_train_fe['Observed Length (m)'] + 1e-6)
    X_valid_fe['Weight_per_Length'] = X_valid_fe['Observed Weight (kg)'] / (X_valid_fe['Observed Length (m)'] + 1e-6)

print(f"Added interaction features. New feature count: {X_train_fe.shape[1]}")

# 2. Target Encoding for High-Cardinality Categorical Features
# Using cross-validation to prevent target leakage
print("\nApplying target encoding to categorical features...")

try:
    from category_encoders import TargetEncoder
    use_target_encoder = True
except ImportError:
    print("category_encoders not available. Using manual target encoding with CV...")
    use_target_encoder = False

if use_target_encoder:
    # Target encode categorical columns using cross-validation
    categorical_cols_to_encode = [col for col in ['Country/Region', 'Habitat Type'] 
                                  if col in X_train_fe.columns]
    
    if categorical_cols_to_encode:
        # Use TargetEncoder with CV to prevent leakage
        target_encoder = TargetEncoder(cols=categorical_cols_to_encode, smoothing=1.0)
        X_train_fe_encoded = target_encoder.fit_transform(X_train_fe, y_train)
        X_valid_fe_encoded = target_encoder.transform(X_valid_fe)
        
        # Keep original columns and add encoded versions with suffix
        for col in categorical_cols_to_encode:
            X_train_fe[f'{col}_target_enc'] = X_train_fe_encoded[col]
            X_valid_fe[f'{col}_target_enc'] = X_valid_fe_encoded[col]
        
        print(f"Target encoded columns: {categorical_cols_to_encode}")
else:
    # Manual target encoding with cross-validation
    from sklearn.model_selection import KFold
    from sklearn.preprocessing import LabelEncoder
    
    categorical_cols_to_encode = [col for col in ['Country/Region', 'Habitat Type'] 
                                  if col in X_train_fe.columns]
    
    if categorical_cols_to_encode:
        # Encode target labels to numeric
        temp_le = LabelEncoder()
        y_train_encoded_for_te = temp_le.fit_transform(y_train)
        global_mean = y_train_encoded_for_te.mean()
        
        # 5-fold CV for target encoding prevents leakage
        # Each fold's encoding uses only training data from other folds
        kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        
        for col in categorical_cols_to_encode:
            # Initialize encoded columns
            X_train_fe[f'{col}_target_enc'] = 0.0
            X_valid_fe[f'{col}_target_enc'] = 0.0
            
            for train_idx, val_idx in kf.split(X_train_fe):
                train_mean = pd.Series(y_train_encoded_for_te[train_idx]).groupby(
                    X_train_fe[col].iloc[train_idx]
                ).mean()
                X_train_fe.iloc[val_idx, X_train_fe.columns.get_loc(f'{col}_target_enc')] = (
                    X_train_fe[col].iloc[val_idx].map(train_mean).fillna(global_mean).values
                )
            
            # Validation set encoded using full training set mean
            train_mean = pd.Series(y_train_encoded_for_te).groupby(X_train_fe[col]).mean()
            X_valid_fe[f'{col}_target_enc'] = X_valid_fe[col].map(train_mean).fillna(global_mean)
        
        print(f"Manually target encoded columns: {categorical_cols_to_encode}")

print(f"\nFinal feature count after engineering: {X_train_fe.shape[1]}")
print("Feature engineering complete. Using X_train_fe and X_valid_fe for enhanced models.")


In [ ]:
# Analyze class distribution to determine if imbalance handling is needed

print("="*70)
print("CLASS IMBALANCE ANALYSIS")
print("="*70)

# Analyze class distribution
class_counts = y_train.value_counts().sort_index()
class_proportions = y_train.value_counts(normalize=True).sort_index()

print("\nClass distribution in training set:")
for cls, count in class_counts.items():
    prop = class_proportions[cls]
    print(f"  {cls}: {count} samples ({prop:.2%})")

# Compute balanced class weights using inverse frequency
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

# Create temp encoder for weight calculation
temp_label_encoder = LabelEncoder()
y_train_for_weights = temp_label_encoder.fit_transform(y_train)
# 'balanced' strategy computes n_samples / (n_classes * np.bincount(y))
# This gives higher weight to minority classes during training
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_for_weights), y=y_train_for_weights)
class_weight_dict = dict(enumerate(class_weights))

print(f"\nComputed class weights (balanced):")
for i, (cls, weight) in enumerate(zip(temp_label_encoder.classes_, class_weights)):
    print(f"  {cls}: {weight:.3f}")

# Check if imbalance is significant
min_prop = class_proportions.min()
max_prop = class_proportions.max()
imbalance_ratio = max_prop / min_prop

print(f"\nImbalance ratio (max/min): {imbalance_ratio:.2f}")
if imbalance_ratio > 2.0:
    print("  -> Significant class imbalance detected. Applying class weights recommended.")
else:
    print("  -> Classes are relatively balanced. Class weights may still help.")

print("\n" + "="*70)
print("Class imbalance analysis complete.")
print("Class weights will be applied to models that support it.")
print("="*70)


In [ ]:
# XGBoost baseline model

# XGBoost needs numeric labels for multi-class classification
# LabelEncoder converts string class names -> ints
from sklearn.preprocessing import LabelEncoder
baseline_label_encoder = LabelEncoder()
y_train_baseline_encoded = baseline_label_encoder.fit_transform(y_train)
y_valid_baseline_encoded = baseline_label_encoder.transform(y_valid)

n_classes_baseline = len(baseline_label_encoder.classes_)

# Baseline hyperparameters chosen based on common practice
xgb_baseline = xgb.XGBClassifier(
    objective='multi:softprob',  
    eval_metric='mlogloss',  
    num_class=n_classes_baseline,
    random_state=RANDOM_STATE,
    n_estimators=100,  
    learning_rate=0.1,  
    max_depth=5,  
    subsample=0.8,  
    colsample_bytree=0.8,  
    tree_method='hist'
)

xgb_baseline_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', xgb_baseline)
])

xgb_baseline_pipeline.fit(X_train, y_train_baseline_encoded)

# Evaluate on validation set
valid_proba = xgb_baseline_pipeline.predict_proba(X_valid)
valid_pred_encoded = xgb_baseline_pipeline.predict(X_valid)
valid_pred = baseline_label_encoder.inverse_transform(valid_pred_encoded)

baseline_logloss = log_loss(y_valid_baseline_encoded, valid_proba)
baseline_acc = accuracy_score(y_valid, valid_pred)

print(f"Baseline XGBoost - Log Loss: {baseline_logloss:.4f}, Accuracy: {baseline_acc:.4f}")
print("Confusion matrix (baseline):")
print(confusion_matrix(y_valid, valid_pred))


In [ ]:
# XGBoost w/ Feature Engineering
# Testing if engineered features improve performance

# Update preprocessor to handle new features
numeric_cols_fe = X_train_fe.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols_fe = X_train_fe.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_transformer_fe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_fe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor_fe = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_fe, numeric_cols_fe),
        ('cat', categorical_transformer_fe, categorical_cols_fe)
    ]
)

# XGBoost w/ feature engineering
xgb_fe = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=n_classes_baseline,
    random_state=RANDOM_STATE,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist'
)

xgb_fe_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor_fe),
    ('model', xgb_fe)
])

xgb_fe_pipeline.fit(X_train_fe, y_train_baseline_encoded)

# Evaluate
valid_proba_fe = xgb_fe_pipeline.predict_proba(X_valid_fe)
valid_pred_fe_encoded = xgb_fe_pipeline.predict(X_valid_fe)
valid_pred_fe = baseline_label_encoder.inverse_transform(valid_pred_fe_encoded)

xgb_fe_logloss = log_loss(y_valid_baseline_encoded, valid_proba_fe)
xgb_fe_acc = accuracy_score(y_valid, valid_pred_fe)

print(f"XGBoost with Feature Engineering - Log Loss: {xgb_fe_logloss:.4f}, Accuracy: {xgb_fe_acc:.4f}")
print("Confusion matrix (XGBoost + FE):")
print(confusion_matrix(y_valid, valid_pred_fe))


In [ ]:
# XGBoost with Class Weights (Handling Imbalance)
# Testing if class weights improve performance on minority classes

xgb_balanced = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=n_classes_baseline,
    random_state=RANDOM_STATE,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',
    # Class weights applied via sample_weight parameter in fit() method
)

xgb_balanced_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', xgb_balanced)
])

# Convert class weights to sample weights for XGBoost
sample_weights_train = np.array([class_weight_dict[y] for y in y_train_baseline_encoded])

xgb_balanced_pipeline.fit(X_train, y_train_baseline_encoded, model__sample_weight=sample_weights_train)

# Evaluate
valid_proba_balanced = xgb_balanced_pipeline.predict_proba(X_valid)
valid_pred_balanced_encoded = xgb_balanced_pipeline.predict(X_valid)
valid_pred_balanced = baseline_label_encoder.inverse_transform(valid_pred_balanced_encoded)

xgb_balanced_logloss = log_loss(y_valid_baseline_encoded, valid_proba_balanced)
xgb_balanced_acc = accuracy_score(y_valid, valid_pred_balanced)

print(f"XGBoost with Class Weights - Log Loss: {xgb_balanced_logloss:.4f}, Accuracy: {xgb_balanced_acc:.4f}")
print("Confusion matrix (XGBoost + Balanced):")
print(confusion_matrix(y_valid, valid_pred_balanced))


In [ ]:
# Optuna Bayesian optimization for XGBoost hyperparameters
# Bayesian optimization (Optuna) chosen over GridSearchCV
# Optuna uses Tree-structured Parzen Estimator (TPE) to explore hyperparameter space

N_TRIALS = 30  # 30 trials balances exploration vs computation time

# Encode labels for XGBoost (needed for num_class parameter)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

n_classes = len(label_encoder.classes_)
print(f"Number of classes: {n_classes}")


def objective(trial):
    # Hyperparameter search space based on XGBoost best practices
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),  
        'max_depth': trial.suggest_int('max_depth', 3, 11),  
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),  
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),  
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),  
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  
        'gamma': trial.suggest_float('gamma', 0.0, 5.0), 
    }

    model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        num_class=n_classes,
        random_state=RANDOM_STATE,
        tree_method='hist',
        **params
    )

    pipe = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('model', model)
    ])

    # 3-fold stratified cross-validation on training set only
    # preserves class distribution in each fold
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    cv_log_losses = []

    for train_idx, valid_idx in skf.split(X_train, y_train_encoded):
        X_tr, X_va = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_va = y_train_encoded[train_idx], y_train_encoded[valid_idx]

        pipe.fit(X_tr, y_tr)
        proba_va = pipe.predict_proba(X_va)
        # Log loss used as optimization metric
        cv_log_losses.append(log_loss(y_va, proba_va))

    return np.mean(cv_log_losses)


# TPE (Tree-structured Parzen Estimator) sampler for Bayesian optimization
study = optuna.create_study(
    direction='minimize',  # minimize log loss
    study_name='xgb_crocodiles_opt',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)

print("Best trial:")
print(f"Best CV log loss: {study.best_trial.value:.4f}")
print("Best parameters:")
print(study.best_trial.params)


In [ ]:
# Train final XGBoost model with best Optuna parameters and evaluate

best_params = study.best_trial.params
best_xgb = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=n_classes,
    random_state=RANDOM_STATE,
    tree_method='hist',
    **best_params
)

xgb_best_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', best_xgb)
])

# Fit on training set, evaluate on validation set (same split as baseline)
xgb_best_pipeline.fit(X_train, y_train_encoded)

valid_proba_best = xgb_best_pipeline.predict_proba(X_valid)
valid_pred_best_encoded = xgb_best_pipeline.predict(X_valid)
valid_pred_best = label_encoder.inverse_transform(valid_pred_best_encoded)

best_logloss = log_loss(y_valid_encoded, valid_proba_best)
best_acc = accuracy_score(y_valid, valid_pred_best)

print(f"Tuned XGBoost - Log Loss: {best_logloss:.4f}, Accuracy: {best_acc:.4f}")
print("Confusion matrix (tuned):")
print(confusion_matrix(y_valid, valid_pred_best))


In [ ]:
# CatBoost Implementation
# CatBoost chosen as alternative gradient boosting algorithm
# Key advantage: native categorical feature handling without one-hot encoding
# This may reduce dimensionality and improve performance on categorical features

try:
    from catboost import CatBoostClassifier
    
    print("="*70)
    print("CATBOOST IMPLEMENTATION")
    print("="*70)
    
    # Identify categorical columns for CatBoost
    cat_features_indices = [i for i, col in enumerate(X_train.columns) 
                            if col in categorical_cols]
    
    print(f"\nCategorical features (indices): {cat_features_indices}")
    print(f"Categorical feature names: {[X_train.columns[i] for i in cat_features_indices]}")
    
    # CatBoost Baseline
    catboost_baseline = CatBoostClassifier(
        iterations=100,
        learning_rate=0.1,
        depth=5,
        loss_function='MultiClass',
        random_seed=RANDOM_STATE,
        verbose=False,
        cat_features=cat_features_indices if cat_features_indices else None
    )
    
    # CatBoost can handle raw DataFrames with categoricals
    catboost_baseline.fit(
        X_train, y_train_baseline_encoded,
        cat_features=cat_features_indices if cat_features_indices else None,
        verbose=False
    )
    
    # Evaluate
    catboost_baseline_proba = catboost_baseline.predict_proba(X_valid)
    catboost_baseline_pred_encoded = catboost_baseline.predict(X_valid)
    catboost_baseline_pred = baseline_label_encoder.inverse_transform(catboost_baseline_pred_encoded)
    
    catboost_baseline_logloss = log_loss(y_valid_baseline_encoded, catboost_baseline_proba)
    catboost_baseline_acc = accuracy_score(y_valid, catboost_baseline_pred)
    
    print(f"\nCatBoost Baseline - Log Loss: {catboost_baseline_logloss:.4f}, Accuracy: {catboost_baseline_acc:.4f}")
    print("Confusion matrix (CatBoost Baseline):")
    print(confusion_matrix(y_valid, catboost_baseline_pred))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")
    catboost_baseline = None
    catboost_baseline_logloss = None
    catboost_baseline_acc = None


In [ ]:
# CatBoost w/ Optuna Hyperparameter Tuning

SKIP_CATBOOST_TUNING = False  # Set to True to skip tuning (use baseline only), cause it can be slow

try:
    from catboost import CatBoostClassifier
    
    if catboost_baseline is not None and not SKIP_CATBOOST_TUNING:
        print("="*70)
        print("CATBOOST OPTUNA TUNING")
        print("="*70)
        print("NOTE: This may take several minutes. Interrupt if needed.")
        print("You can skip this by setting SKIP_CATBOOST_TUNING = True above.")
        print("="*70)
        
        # Same num of trials as XGBoost for fair comparison
        N_TRIALS_CAT = 30
        # Limited iterations during tuning to reduce computation time
        MAX_ITERATIONS = 50
        
        def catboost_objective(trial):
            params = {
                'iterations': trial.suggest_int('iterations', 30, MAX_ITERATIONS),
                'depth': trial.suggest_int('depth', 3, 7),
                'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 5),
                'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
                'random_strength': trial.suggest_float('random_strength', 0.5, 1.5),
            }
            
            if params['bootstrap_type'] == 'Bayesian':
                params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.5, 1.0)
            elif params['bootstrap_type'] == 'Bernoulli':
                params['subsample'] = trial.suggest_float('subsample', 0.7, 1.0)
            
            # 2-fold CV used instead of 3-fold to reduce computation time
            skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
            cv_log_losses = []
            
            # Ensure y is 1D array
            y_train_flat = np.array(y_train_baseline_encoded).ravel()
            
            for train_idx, valid_idx in skf.split(X_train, y_train_flat):
                X_tr, X_va = X_train.iloc[train_idx], X_train.iloc[valid_idx]
                y_tr = y_train_flat[train_idx]
                y_va = y_train_flat[valid_idx]
                
                # Create a fresh model for each fold
                fold_model = CatBoostClassifier(
                    loss_function='MultiClass',
                    random_seed=RANDOM_STATE,
                    verbose=False,
                    cat_features=cat_features_indices if cat_features_indices else None,
                    early_stopping_rounds=5,
                    iterations=params['iterations'], 
                    **{k: v for k, v in params.items() if k != 'iterations'}
                )
                
                fold_model.fit(X_tr, y_tr, verbose=False)
                proba_va = fold_model.predict_proba(X_va)
                cv_log_losses.append(log_loss(y_va, proba_va))
            
            return np.mean(cv_log_losses)
        
        try:
            # Same sampler and seed as XGBoost for reproducibility and fair comparison
            catboost_study = optuna.create_study(
                direction='minimize', 
                study_name='catboost_crocodiles_opt',
                sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
            )
            catboost_study.optimize(catboost_objective, n_trials=N_TRIALS_CAT, show_progress_bar=False)
            
            print("Best trial:")
            print(f"Best CV log loss: {catboost_study.best_trial.value:.4f}")
            print("Best parameters:")
            print(catboost_study.best_trial.params)
            
            # Train final CatBoost model with best parameters
            best_catboost_params = catboost_study.best_trial.params.copy()
            # Handle bootstrap-specific params
            bootstrap_type = best_catboost_params.pop('bootstrap_type', 'Bayesian')
            if 'bagging_temperature' in best_catboost_params:
                best_catboost_params['bootstrap_type'] = 'Bayesian'
            elif 'subsample' in best_catboost_params:
                best_catboost_params['bootstrap_type'] = 'Bernoulli'
            else:
                best_catboost_params['bootstrap_type'] = bootstrap_type
            
            # Limit final model iterations too
            if 'iterations' in best_catboost_params:
                best_catboost_params['iterations'] = min(best_catboost_params['iterations'], 100)
            
            catboost_tuned = CatBoostClassifier(
                loss_function='MultiClass',
                random_seed=RANDOM_STATE,
                verbose=False,
                cat_features=cat_features_indices if cat_features_indices else None,
                early_stopping_rounds=10,
                **best_catboost_params
            )
            
            # Ensure y is 1D array
            y_train_catboost = np.array(y_train_baseline_encoded).ravel()
            catboost_tuned.fit(X_train, y_train_catboost, verbose=False)
            
            # Evaluate
            catboost_tuned_proba = catboost_tuned.predict_proba(X_valid)
            catboost_tuned_pred_encoded = catboost_tuned.predict(X_valid)
            catboost_tuned_pred = baseline_label_encoder.inverse_transform(catboost_tuned_pred_encoded)
            
            catboost_tuned_logloss = log_loss(y_valid_baseline_encoded, catboost_tuned_proba)
            catboost_tuned_acc = accuracy_score(y_valid, catboost_tuned_pred)
            
            print(f"\nCatBoost Tuned - Log Loss: {catboost_tuned_logloss:.4f}, Accuracy: {catboost_tuned_acc:.4f}")
            print("Confusion matrix (CatBoost Tuned):")
            print(confusion_matrix(y_valid, catboost_tuned_pred))
        except KeyboardInterrupt:
            print("\nCatBoost tuning interrupted. Using baseline only.")
            catboost_tuned_logloss = None
            catboost_tuned_acc = None
    elif SKIP_CATBOOST_TUNING:
        print("CatBoost tuning skipped (SKIP_CATBOOST_TUNING = True). Using baseline only.")
        catboost_tuned_logloss = None
        catboost_tuned_acc = None
    else:
        print("CatBoost baseline not available. Skipping tuning.")
        catboost_tuned_logloss = None
        catboost_tuned_acc = None
        
except (ImportError, NameError):
    print("CatBoost not available. Skipping tuning.")
    catboost_tuned_logloss = None
    catboost_tuned_acc = None


In [ ]:
# Feature Importance Analysis
# Compare feature importances across different models to understand what drives predictions

print("="*70)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*70)

# Get feature names from preprocessor
preprocessor_fitted = xgb_best_pipeline.named_steps['preprocess']
feature_names = []

# Extract feature names from ColumnTransformer
if hasattr(preprocessor_fitted, 'transformers_'):
    for name, transformer, cols in preprocessor_fitted.transformers_:
        if name == 'num':
            feature_names.extend(cols)
        elif name == 'cat':
            # For one-hot encoded features, get the feature names
            if hasattr(transformer.named_steps['onehot'], 'get_feature_names_out'):
                cat_feature_names = transformer.named_steps['onehot'].get_feature_names_out(cols)
                feature_names.extend(cat_feature_names)
            else:
                # Fallback: use column names
                feature_names.extend([f"{col}_{i}" for i, col in enumerate(cols)])

# XGBoost Feature Importance
xgb_tuned_model = xgb_best_pipeline.named_steps['model']
xgb_importance = xgb_tuned_model.feature_importances_

# Create DataFrame for easier visualization
importance_df = pd.DataFrame({
    'Feature': feature_names[:len(xgb_importance)],
    'XGBoost_Tuned_Importance': xgb_importance
}).sort_values('XGBoost_Tuned_Importance', ascending=False)

# Also get baseline XGBoost importance for comparison
xgb_baseline_model = xgb_baseline_pipeline.named_steps['model']
xgb_baseline_importance = xgb_baseline_model.feature_importances_
importance_df['XGBoost_Baseline_Importance'] = xgb_baseline_importance[:len(importance_df)]

# Get top 15 features
top_features = importance_df.head(15)

print("\nTop 15 Most Important Features (Tuned XGBoost):")
print(top_features[['Feature', 'XGBoost_Tuned_Importance']].to_string(index=False))

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Plot 1: Top features for tuned XGBoost
axes[0].barh(range(len(top_features)), top_features['XGBoost_Tuned_Importance'].values[::-1])
axes[0].set_yticks(range(len(top_features)))
axes[0].set_yticklabels(top_features['Feature'].values[::-1], fontsize=9)
axes[0].set_xlabel('Feature Importance', fontsize=11)
axes[0].set_title('Top 15 Features - XGBoost Tuned', fontsize=12, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Plot 2: Comparison between baseline and tuned
top_10 = importance_df.head(10)
x_pos = np.arange(len(top_10))
width = 0.35

axes[1].barh(x_pos - width/2, top_10['XGBoost_Baseline_Importance'].values, 
              width, label='Baseline', alpha=0.8)
axes[1].barh(x_pos + width/2, top_10['XGBoost_Tuned_Importance'].values, 
              width, label='Tuned', alpha=0.8)
axes[1].set_yticks(x_pos)
axes[1].set_yticklabels(top_10['Feature'].values, fontsize=9)
axes[1].set_xlabel('Feature Importance', fontsize=11)
axes[1].set_title('Feature Importance: Baseline vs Tuned', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# CatBoost feature importance if available
if 'catboost_tuned' in locals() and catboost_tuned is not None:
    print("\n" + "="*70)
    print("CATBOOST FEATURE IMPORTANCE")
    print("="*70)
    
    catboost_importance = catboost_tuned.get_feature_importance()
    # CatBoost uses original feature indices, need to map to feature names
    catboost_importance_df = pd.DataFrame({
        'Feature': [X_train.columns[i] if i < len(X_train.columns) else f'Feature_{i}' 
                   for i in range(len(catboost_importance))],
        'CatBoost_Importance': catboost_importance
    }).sort_values('CatBoost_Importance', ascending=False)
    
    print("\nTop 15 Most Important Features (CatBoost Tuned):")
    print(catboost_importance_df.head(15)[['Feature', 'CatBoost_Importance']].to_string(index=False))
    
    # Plot CatBoost importance
    plt.figure(figsize=(10, 8))
    top_catboost = catboost_importance_df.head(15)
    plt.barh(range(len(top_catboost)), top_catboost['CatBoost_Importance'].values[::-1])
    plt.yticks(range(len(top_catboost)), top_catboost['Feature'].values[::-1], fontsize=9)
    plt.xlabel('Feature Importance', fontsize=11)
    plt.title('Top 15 Features - CatBoost Tuned', fontsize=12, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

print("\n" + "="*70)
print("Feature importance analysis complete.")
print("="*70)


In [ ]:
# Analyze how confident the models are in their predictions

print("="*70)
print("PREDICTION CONFIDENCE ANALYSIS")
print("="*70)

# Get prediction probabilities for all models
models_to_analyze = []

# XGBoost Baseline
if 'xgb_baseline_pipeline' in locals():
    xgb_baseline_proba = xgb_baseline_pipeline.predict_proba(X_valid)
    models_to_analyze.append(('XGBoost Baseline', xgb_baseline_proba))

# XGBoost Tuned
if 'xgb_best_pipeline' in locals():
    xgb_tuned_proba = xgb_best_pipeline.predict_proba(X_valid)
    models_to_analyze.append(('XGBoost Tuned', xgb_tuned_proba))

# CatBoost Baseline
if 'catboost_baseline' in locals() and catboost_baseline is not None:
    catboost_baseline_proba = catboost_baseline.predict_proba(X_valid)
    models_to_analyze.append(('CatBoost Baseline', catboost_baseline_proba))

# CatBoost Tuned
if 'catboost_tuned' in locals() and catboost_tuned is not None:
    catboost_tuned_proba = catboost_tuned.predict_proba(X_valid)
    models_to_analyze.append(('CatBoost Tuned', catboost_tuned_proba))

# Calculate max probability (confidence) for each prediction
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (model_name, proba) in enumerate(models_to_analyze[:4]):
    max_proba = np.max(proba, axis=1)
    
    axes[idx].hist(max_proba, bins=20, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel('Maximum Prediction Probability (Confidence)', fontsize=10)
    axes[idx].set_ylabel('Number of Samples', fontsize=10)
    axes[idx].set_title(f'{model_name}\nMean Confidence: {max_proba.mean():.3f}', fontsize=11, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].axvline(max_proba.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {max_proba.mean():.3f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

# Summary statistics
print("\nPrediction Confidence Summary:")
print("-" * 70)
for model_name, proba in models_to_analyze:
    max_proba = np.max(proba, axis=1)
    print(f"{model_name:20s} | Mean: {max_proba.mean():.4f} | Std: {max_proba.std():.4f} | Min: {max_proba.min():.4f} | Max: {max_proba.max():.4f}")

print("\n" + "="*70)


In [ ]:
# Comprehensive Model Comparison and Interpretation

print("="*70)
print("COMPREHENSIVE MODEL PERFORMANCE SUMMARY")
print("="*70)

# Build results list
results_list = [
    {
        'Model': 'XGBoost Baseline',
        'Log Loss': baseline_logloss,
        'Accuracy': baseline_acc,
        'Notes': 'Original baseline model'
    },
    {
        'Model': 'XGBoost Tuned (Optuna)',
        'Log Loss': best_logloss,
        'Accuracy': best_acc,
        'Notes': 'Bayesian optimized hyperparameters'
    }
]

if 'xgb_fe_logloss' in locals() and xgb_fe_logloss is not None:
    results_list.append({
        'Model': 'XGBoost + Feature Engineering',
        'Log Loss': xgb_fe_logloss,
        'Accuracy': xgb_fe_acc,
        'Notes': 'With interaction features & target encoding'
    })

if 'xgb_balanced_logloss' in locals() and xgb_balanced_logloss is not None:
    results_list.append({
        'Model': 'XGBoost + Class Weights',
        'Log Loss': xgb_balanced_logloss,
        'Accuracy': xgb_balanced_acc,
        'Notes': 'With balanced class weights'
    })

if 'catboost_baseline_logloss' in locals() and catboost_baseline_logloss is not None:
    results_list.append({
        'Model': 'CatBoost Baseline',
        'Log Loss': catboost_baseline_logloss,
        'Accuracy': catboost_baseline_acc,
        'Notes': 'Native categorical handling'
    })

if 'catboost_tuned_logloss' in locals() and catboost_tuned_logloss is not None:
    results_list.append({
        'Model': 'CatBoost Tuned (Optuna)',
        'Log Loss': catboost_tuned_logloss,
        'Accuracy': catboost_tuned_acc,
        'Notes': 'Bayesian optimized'
    })


results_summary = pd.DataFrame(results_list)

print("\nResults Table (sorted by Accuracy):")
results_sorted = results_summary.sort_values('Accuracy', ascending=False)
print(results_sorted.to_string(index=False))

print("\n" + "="*70)
print("KEY OBSERVATIONS")
print("="*70)

# Find best models
best_acc_model = results_summary.loc[results_summary['Accuracy'].idxmax()]
best_logloss_model = results_summary.loc[results_summary['Log Loss'].idxmin()]

print(f"\n1. Best Accuracy: {best_acc_model['Model']} ({best_acc_model['Accuracy']:.2%})")
print(f"2. Best Log Loss: {best_logloss_model['Model']} ({best_logloss_model['Log Loss']:.4f})")

print(f"\n3. XGBoost Baseline: {baseline_acc:.2%} accuracy, {baseline_logloss:.4f} log loss")
print(f"4. XGBoost Tuned: {best_acc:.2%} accuracy, {best_logloss:.4f} log loss")
if 'xgb_fe_logloss' in locals() and xgb_fe_logloss is not None:
    print(f"5. XGBoost + FE: {xgb_fe_acc:.2%} accuracy, {xgb_fe_logloss:.4f} log loss")
    fe_improvement = xgb_fe_acc - baseline_acc
    if fe_improvement > 0:
        print(f"   -> Feature engineering improved accuracy by {fe_improvement:.2%}")
    else:
        print(f"   -> Feature engineering decreased accuracy by {abs(fe_improvement):.2%}")

if 'catboost_baseline_logloss' in locals() and catboost_baseline_logloss is not None:
    print(f"6. CatBoost Baseline: {catboost_baseline_acc:.2%} accuracy, {catboost_baseline_logloss:.4f} log loss")
    catboost_vs_xgb = catboost_baseline_acc - baseline_acc
    if catboost_vs_xgb > 0:
        print(f"   -> CatBoost outperforms XGBoost baseline by {catboost_vs_xgb:.2%}")
    else:
        print(f"   -> XGBoost baseline outperforms CatBoost by {abs(catboost_vs_xgb):.2%}")

print("\n" + "="*70)
print("RECOMMENDATIONS")
print("="*70)
print("Based on the results, use the model with the best balance of accuracy and log loss.")
print("If accuracy drops with new features, revert to the baseline XGBoost model.")
print("="*70)
